## Load module and libraries

In [17]:
from selenium import webdriver
from datetime import datetime, timedelta
from random import randrange
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import calendar
import json
import time
import os

## Download login info as cookie (run this only when you need cookie)

In [2]:
# ## Open the virtual browser 
# driver = webdriver.Firefox()

# ## Open the website 
# website = 'https://enlighten.enphaseenergy.com/systems/1302574/inverters/28563594/time_series_x?&date=2019-05-04&stat=POWR%2CDCV%2CDCA%2CACV%2CACHZ%2CTMPI'
# driver.get(website)

# ## Username and password
# user="mfeng@umass.edu"
# password="UmassCS!"

# ## Login the website
# driver.find_element_by_id('user_email').click()
# driver.find_element_by_id("user_email").send_keys(user)
# driver.find_element_by_id('user_password').click()
# driver.find_element_by_id("user_password").send_keys(password)
# driver.find_element_by_id('submit').click()

# ## Store the login info in cookie
# driver.get(website)
# cookie_items = driver.get_cookies()

# post = {}

# for cookie_item in cookie_items:
#     post[cookie_item['name']] = cookie_item['value']
    
# cookie_str = json.dumps(post)
# with open('cookie.txt', 'w', encoding='utf-8') as f:
#     f.write(cookie_str)
# f.close

## Download data for each panel

In [18]:
# Use the stored cookie to login and keep session open
with open('cookie.txt', 'r',encoding='utf-8') as f:
    cookie = f.read()
cookies = json.loads(cookie)

In [19]:
## Align microinverter's serial number with inverter ID
system_id = '1302574'

## Inverter serial number and ID is aligned one by one 
inverter_sn = ['121721038143', '121721037801', '121721037691', '121721038936', '121721038148', 
             '121721037892', '121721037821', '121721037806', '121721038079', '121721038122', 
             '121721038133', '121721037662', '121721037689', '121721037871', '121721037817', 
             '121721038020', '121721038147', '121721038076', '121721038911', '121721037842', 
             '121721037788', '121721037686', '121721037867', '121721038108', '121721038125', 
             '121721037685', '121721038107', '121721038154', '121721038144', '121721037880', 
             '121721038037']

inverter_id = [str(28563593+i) for i in range(len(inverter_sn))]
inverter_sn_id = {k:v for k,v in zip(inverter_id,inverter_sn)}

In [20]:
print("Microinverter ID: serial number")
inverter_sn_id

Microinverter ID: serial number


{'28563593': '121721038143',
 '28563594': '121721037801',
 '28563595': '121721037691',
 '28563596': '121721038936',
 '28563597': '121721038148',
 '28563598': '121721037892',
 '28563599': '121721037821',
 '28563600': '121721037806',
 '28563601': '121721038079',
 '28563602': '121721038122',
 '28563603': '121721038133',
 '28563604': '121721037662',
 '28563605': '121721037689',
 '28563606': '121721037871',
 '28563607': '121721037817',
 '28563608': '121721038020',
 '28563609': '121721038147',
 '28563610': '121721038076',
 '28563611': '121721038911',
 '28563612': '121721037842',
 '28563613': '121721037788',
 '28563614': '121721037686',
 '28563615': '121721037867',
 '28563616': '121721038108',
 '28563617': '121721038125',
 '28563618': '121721037685',
 '28563619': '121721038107',
 '28563620': '121721038154',
 '28563621': '121721038144',
 '28563622': '121721037880',
 '28563623': '121721038037'}

In [21]:
## Group panels based on their facing direction (west/east/lower)
west_roof = ['121721038107', '121721037685', '121721038037', '121721038147', '121721037806', 
             '121721037892', '121721038143']
east_roof = ['121721037871', '121721037662', '121721037801', '121721037691', '121721037880',
             '121721038936', '121721038122', '121721038148', '121721038133']
lower_roof = ['121721037686', '121721037788', '121721038076', '121721038079', '121721037867', 
              '121721038020', '121721037817', '121721038144', '121721038911', '121721037842']

west_roof_id = []
east_roof_id = []
lower_roof_id = []

for inverter in inverter_sn:
    if inverter in west_roof:
        west_roof_id.append(inverter_id[inverter_sn.index(inverter)])
    elif inverter in east_roof:
        east_roof_id.append(inverter_id[inverter_sn.index(inverter)])
    elif inverter in lower_roof:
        lower_roof_id.append(inverter_id[inverter_sn.index(inverter)])

In [22]:
## Generate calendar list by defining start and end date

start_day = '2017-9-11'
end_day = '2019-11-11'
start_day = datetime.strptime(start_day, '%Y-%m-%d')
end_day = datetime.strptime(end_day, '%Y-%m-%d')
total_day = abs((end_day - start_day).days)


def calendar_test(day, count, date_list = None):
    if date_list is None:
        date_list = []
    if count == 0:
        return date_list
    count -= 1
    day = day + timedelta(days=1)
    date = day.isoformat().split('T')[0]
    date_list.append(date)
    
    calendar_test(day,count,date_list)
    
    return date_list

day_list = calendar_test(start_day,total_day)

In [23]:
## Formula: website = link_1 + system_id + link_2 + inverter_id + link_3 + date + link_4
link_1 = 'https://enlighten.enphaseenergy.com/systems/'
link_2 = '/inverters/'
link_3 = '/time_series_x?&date='
link_4 = '&stat=POWR%2CDCV%2CDCA%2CACV%2CACHZ%2CTMPI'

In [24]:
# Define which direction of roof panels you'd like to collect from
roof_panel = east_roof_id

In [25]:
for inverter_id in roof_panel:
    print('Inverter ID: {}'.format(inverter_id))
    
    info = {'date_time':[], 'power':[], 'power_unknown':[], 'DCV':[], 
            'DCA':[], 'ACV':[], 'ACHZ':[], 'TMPI':[]}
    for day in day_list:
        # website address where data is stored
        website = link_1 + system_id + link_2 + inverter_id + link_3 + day + link_4
        # access data in json format from the website
        res = requests.get(url=website, cookies=cookies)
        data = res.json()
        
        # extract time information 
        for ii in data['POWR']:
            date_time = time.struct_time(time.localtime(ii[0]))
            date = 10000*date_time[0]+100*date_time[1]+1*date_time[2]
            time_ = 100*date_time[3]+1*date_time[4]
            date_time = date*10000+time_
            
            info['date_time'].append(date_time)
            info['power'].append(ii[1])
            info['power_unknown'].append(ii[2])
                        
        for ii in data['DCV']:
            info['DCV'].append(ii[1])
            
        for ii in data['DCA']:
            info['DCA'].append(ii[1])
            
        for ii in data['ACV']:
            info['ACV'].append(ii[1])
            
        for ii in data['ACHZ']:
            info['ACHZ'].append(ii[1])
            
        for ii in data['TMPI']:
            info['TMPI'].append(ii[1])
            
    # Save downloaded data for each panel
    data = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in info.items() ]))
    export_csv = data.to_csv (str(inverter_id)+".csv", index = True, header=True)

Inverter ID: 28563594
Inverter ID: 28563595
Inverter ID: 28563596
Inverter ID: 28563597
Inverter ID: 28563602
Inverter ID: 28563603
Inverter ID: 28563604
Inverter ID: 28563606
Inverter ID: 28563622


## Clean Data 

In [26]:
# Load files from current directory
files = os.listdir(os.getcwd())
files = [file for file in files if file[:-4] in roof_panel]

In [27]:
def load_data(file):
    
    ## Load dataset 
    dataset = pd.read_csv(file, index_col=0)
    dataset = dataset.set_index('date_time')
    dataset = dataset.drop(['power_unknown', 'DCV', 'DCA', 'ACV', 'ACHZ', 'TMPI'], axis=1)
    dataset = dataset.rename(columns = {'power': str(file[:-4])})
    
    return dataset

In [28]:
dataset = [load_data(file) for file in files]

In [29]:
# Merge power output for each panel based on timestamp
data = dataset[0]
for i in range(1, len(dataset)):
#     data = pd.merge(data, dataset[i], left_index=True, right_index=True)
    data = data.join(dataset[i], lsuffix='_caller', rsuffix='_other')
    data = data.interpolate(method ='linear', limit_direction ='both', limit = 3) 

# Remove duplicated row   
data = data.reset_index().drop_duplicates(subset='date_time',keep='first').set_index('date_time')
data = data.dropna()

In [30]:
data

,28563595,28563594,28563596,28563622,28563597,28563606,28563604,28563602,28563603
date_time,,,,,,,,,
201709120640,0.910,0.010,2.629,2.589,0.752,0.0110,2.453,2.586,0.0100
201709120642,1.545,0.670,2.629,2.589,0.876,1.2815,2.453,2.586,1.3365
201709120649,2.182,1.330,2.629,2.589,1.000,2.5520,2.453,2.586,2.6630
201709120650,2.729,1.000,2.864,2.857,1.017,2.8800,2.720,2.880,2.9610
201709120651,2.982,1.037,3.093,3.123,1.018,3.0180,2.982,3.036,3.1640
...,...,...,...,...,...,...,...,...,...
201911111451,15.335,15.601,15.872,21.749,16.108,15.4650,15.898,15.822,15.8310
201911111506,19.489,19.697,20.020,21.749,20.259,19.2980,19.895,19.765,19.7500
201911111521,10.193,10.362,10.410,21.749,10.625,10.1310,10.435,10.426,10.3520


In [31]:
# Save the cleaned data
export_csv = data.to_csv ("cleaned_data.csv", index = True, header=True)